In [8]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [9]:
from datetime import datetime, timedelta
import pandas as pd

current_date = pd.to_datetime(datetime.utcnow()).floor('h')
print(f'{current_date=}')

current_date=Timestamp('2025-04-14 03:00:00')


In [10]:
from src.inference import load_batch_of_features_from_store

features = load_batch_of_features_from_store(current_date)

2025-04-13 23:49:16,104 INFO: Closing external client and cleaning up certificates.
Connection closed.
2025-04-13 23:49:16,108 INFO: Initializing external client
2025-04-13 23:49:16,109 INFO: Base URL: https://c.app.hopsworks.ai:443




To ensure compatibility please install the latest bug fix release matching the minor version of your backend (4.2) by running 'pip install hopsworks==4.2.*'


2025-04-13 23:49:16,624 INFO: Python Engine initialized.

Logged in to project, explore it here https://c.app.hopsworks.ai:443/p/1211558
Fetching data from 2025-03-17 03:00:00 to 2025-04-14 02:00:00
Finished: Reading data from Hopsworks, using Hopsworks Feature Query Service (10.36s) 


In [11]:
from src.inference import (
    load_model_from_registry,
    get_model_predictions
)

model = load_model_from_registry()
predictions = get_model_predictions(model, features)
print(predictions)

2025-04-13 23:49:29,883 INFO: Closing external client and cleaning up certificates.
Connection closed.
2025-04-13 23:49:29,887 INFO: Initializing external client
2025-04-13 23:49:29,887 INFO: Base URL: https://c.app.hopsworks.ai:443




To ensure compatibility please install the latest bug fix release matching the minor version of your backend (4.2) by running 'pip install hopsworks==4.2.*'


2025-04-13 23:49:30,411 INFO: Python Engine initialized.

Logged in to project, explore it here https://c.app.hopsworks.ai:443/p/1211558
[LightGBM] [Warning] feature_fraction is set=0.7167492637678337, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=0.7167492637678337
[LightGBM] [Warning] bagging_fraction is set=0.4029105817023335, subsample=1.0 will be ignored. Current value: bagging_fraction=0.4029105817023335
     pickup_location_id  predicted_demand
0                     1              14.0
1                     2              12.0
2                     3              14.0
3                     4              11.0
4                     5              12.0
..                  ...               ...
260                 261              11.0
261                 262              12.0
262                 263              10.0
263                 264              11.0
264                 265              10.0

[265 rows x 2 columns]


In [12]:
predictions['pickup_hour'] = current_date
predictions

,pickup_location_id,predicted_demand,pickup_hour
0,1,14.0,2025-04-14 03:00:00
1,2,12.0,2025-04-14 03:00:00
2,3,14.0,2025-04-14 03:00:00
3,4,11.0,2025-04-14 03:00:00
4,5,12.0,2025-04-14 03:00:00
...,...,...,...
260,261,11.0,2025-04-14 03:00:00
261,262,12.0,2025-04-14 03:00:00
262,263,10.0,2025-04-14 03:00:00
263,264,11.0,2025-04-14 03:00:00


## Save these predictions in the feature store, so they can be later consumed by our streamlit app

In [ ]:
from src.feature_store_api import get_feature_store
import src.config as config

# connect to the feature group
feature_group = get_feature_store().get_or_create_feature_group(
    name=config.FEATURE_GROUP_MODEL_PREDICTIONS,
    version=1,
    description="Predictions generate by our production model",
    primary_key = ['pickup_location_id', 'pickup_hour'],
    event_time='pickup_hour',
)

In [ ]:
feature_group.insert(predictions, write_options={"wait_for_job": False})